<a href="https://colab.research.google.com/github/kkkkkkkm/Torch/blob/main/RNN/torch_longSeq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.") #데이터

In [3]:
print(sentence)
print(len(sentence))

if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
180


In [4]:
char_set = list(set(sentence))
print(char_set)

['f', 's', 'y', 'w', 'e', ',', 't', 'c', 'n', 'a', 'i', 'r', 'p', 'g', 'b', '.', 'l', 'o', "'", 'u', 'h', ' ', 'k', 'm', 'd']


In [5]:
char_dic = {c: i for i, c in enumerate(char_set)}
print(char_dic)

{'f': 0, 's': 1, 'y': 2, 'w': 3, 'e': 4, ',': 5, 't': 6, 'c': 7, 'n': 8, 'a': 9, 'i': 10, 'r': 11, 'p': 12, 'g': 13, 'b': 14, '.': 15, 'l': 16, 'o': 17, "'": 18, 'u': 19, 'h': 20, ' ': 21, 'k': 22, 'm': 23, 'd': 24}


In [6]:
dic_size = len(char_dic) 
print(f"사전 크기 {dic_size}") #사전크기, 총 label의 개 수 

사전 크기 25


In [7]:
hidden_size = dic_size
sequence_length = 10
learning_rate = 0.1

In [8]:
#데이터 구성
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):   #sequence_length 길이로 자를 예정, 0~10, ~ 170~180 
  x_str = sentence[i: i+sequence_length] #문자열 저장이 list형태 x_str도 list 형태
  y_str = sentence[i+1 : i+1 + sequence_length]

  print(i, x_str ,'->', y_str)

  x_data.append([char_dic[c] for c in x_str]) #x_str list에서 글자를 꺼내 char_dic에 넣어 글자에 대응 되는 index 뽑아오기
  y_data.append([char_dic[c] for c in y_str]) #위와 같은 형태로 label 설정

#170개의 데이터 생성

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [9]:
print(x_data)

[[10, 0, 21, 2, 17, 19, 21, 3, 9, 8], [0, 21, 2, 17, 19, 21, 3, 9, 8, 6], [21, 2, 17, 19, 21, 3, 9, 8, 6, 21], [2, 17, 19, 21, 3, 9, 8, 6, 21, 6], [17, 19, 21, 3, 9, 8, 6, 21, 6, 17], [19, 21, 3, 9, 8, 6, 21, 6, 17, 21], [21, 3, 9, 8, 6, 21, 6, 17, 21, 14], [3, 9, 8, 6, 21, 6, 17, 21, 14, 19], [9, 8, 6, 21, 6, 17, 21, 14, 19, 10], [8, 6, 21, 6, 17, 21, 14, 19, 10, 16], [6, 21, 6, 17, 21, 14, 19, 10, 16, 24], [21, 6, 17, 21, 14, 19, 10, 16, 24, 21], [6, 17, 21, 14, 19, 10, 16, 24, 21, 9], [17, 21, 14, 19, 10, 16, 24, 21, 9, 21], [21, 14, 19, 10, 16, 24, 21, 9, 21, 1], [14, 19, 10, 16, 24, 21, 9, 21, 1, 20], [19, 10, 16, 24, 21, 9, 21, 1, 20, 10], [10, 16, 24, 21, 9, 21, 1, 20, 10, 12], [16, 24, 21, 9, 21, 1, 20, 10, 12, 5], [24, 21, 9, 21, 1, 20, 10, 12, 5, 21], [21, 9, 21, 1, 20, 10, 12, 5, 21, 24], [9, 21, 1, 20, 10, 12, 5, 21, 24, 17], [21, 1, 20, 10, 12, 5, 21, 24, 17, 8], [1, 20, 10, 12, 5, 21, 24, 17, 8, 18], [20, 10, 12, 5, 21, 24, 17, 8, 18, 6], [10, 12, 5, 21, 24, 17, 8, 18, 6,

In [10]:
print("x_data", x_data[0])
print("y_data", y_data[0])

x_data [10, 0, 21, 2, 17, 19, 21, 3, 9, 8]
y_data [0, 21, 2, 17, 19, 21, 3, 9, 8, 6]


In [11]:
x_one_hot = [np.eye(dic_size)[i] for i in x_data]
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  


In [12]:
print(f"훈련 데이터 크기 {X.shape}")
print(f"레이블 크기 {Y.shape}")

훈련 데이터 크기 torch.Size([170, 10, 25])
레이블 크기 torch.Size([170, 10])


In [13]:
print(X[0])
print(Y[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,

In [27]:
class RNN(nn.Module):
  def __init__(self, input_dim, hidden_size, layers):
    super(RNN, self).__init__()

    self.rnn = nn.RNN(input_dim, hidden_size, num_layers = layers, batch_first = True)
    self.fc = nn.Linear(hidden_size, hidden_size, bias= True)
  
  def forward(self, x):
    out, _states = self.rnn(x)
    out = self.fc(out)


    return out

In [28]:
model = RNN(dic_size, hidden_size, 2)

In [29]:
criterion = torch.nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(model.parameters(), lr = learning_rate)

In [30]:
outputs = model(X)

print(outputs.shape)


torch.Size([170, 10, 25])


In [31]:
print(outputs.view(-1, dic_size).shape) #모든 시점의 값을 Fully connected 층으로 연결

torch.Size([1700, 25])


In [32]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


In [33]:
for i in range(100):
  optimizer.zero_grad()
  outputs = model(X)

  loss = criterion(outputs.view(-1, dic_size), Y.view(-1)) #batch 차원 제거를 위해.

  loss.backward()
  optimizer.step()

  results = outputs.argmax(dim=2)

  predict_str = ""

  for j , result in enumerate(results):
    if j == 0: #처음에는 모든 예측값 가져옴
      predict_str += ''.join([char_set[t] for t in result])
    else:
      predict_str += char_set[result[-1]]

  print(predict_str)

aaaa.,a.u.aaa.aa.a.aauaaaaa.a....aa.faaa.aa.....uf..,uaaafaa.aa.p.uf.a.....u..au.a.aaa.aaaa.aaap.au.p.a.u.a..f..aa.aaauaaafaauaaaaaaap.a.a..a.aa.paauaa.u.uaaaaaaaua.paaa.a.aaaa.pu
oci iiiciiii iiciiiii iiiiiii iiici iiiiiii iiiiii iiiiiiiiiiiciiiiiii iiii iiiiiiiii iiiiiiciiiiiiiiii iii iiiii iii iiiiiiiiiiii iiiiiiiciiiiiiiiciiiiiiciiiiiiiiiiiiiiciiiiiiiii
oooo ooooo ooooooooooooooooooooototoooooooooooooooooooooooooooooooooooooooooooooootooo oooooooooooooooooooooootooooooooooootooooooooooooooooooooootoootoooooootooooooooooooooootooo
 'g  'd b''l.'gll.g'l''l..'l'ld'wl''dl.'..'''.b'lb'llg'l''ll'''l''dl.'l.''l'''l''''lb''l''.'ll'gll''lll''ll.''glc''l''d'b'.b'l'wll'gl''.''ll''bl''ll'g'l''dl''ll''''l''l''''.'''.l'
e dada  dhta la e hla thkaah' ka  ' ka ed ' ld d a la a aaa  ' la da l ha ahhaa lat lhsa  dahla ' ed  l saa aaa rhaaa k  ada  a aa ga lhh' la g ha   aah  daaaa aaaaaaaaa aa 'aaaaa
  e  t th  a hnut   ant tt  n   t a    t en d e ashh   an a tn   ena nnhntt   adnnn an   t e a  tnt 

In [34]:
results = outputs.argmax(dim= 2)

In [25]:
results

tensor([[12, 21,  2,  ...,  9,  8,  6],
        [21,  2, 17,  ...,  8,  6, 21],
        [ 6, 17, 19,  ...,  6, 21,  6],
        ...,
        [21, 17,  0,  ..., 21,  1,  4],
        [ 6,  0, 21,  ...,  1,  4,  9],
        [ 8, 21,  6,  ...,  4,  9, 15]])